In [2]:
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
import seaborn as sns
import numpy as np

# without solve_ivp

In [10]:
def objective(w, tau, S, L, I, I_hat, A, t, delta_x, delta_t):
   
    # Organize your data
    b, d, B, Y, a, e, diffusion_rate = w

    # SLI-model step
    factor = diffusion_rate * delta_t / delta_x**2
    for n in range(0, t.size - 1):
        N = S[:, n] + L[:, n] + I[:, n]
        S[:, n+1] = S[:, n] + delta_t * (b * N - d * S[:, n] - Y * S[:, n] * N - B * S[:, n] * I[:, n])
        L[:, n+1] = L[:, n] + delta_t * (B * S[:, n] * I[:, n] - d * L[:, n] - Y * L[:, n] * N - e * L[:, n])
        I[:, n+1] = I[:, n] + delta_t * (e * L[:, n] - Y * I[:, n] * N - a * I[:, n] - d * I[:, n])

        S[:, n+1] += (factor * A).dot(S[:, n])
        L[:, n+1] += (factor * A).dot(L[:, n])
        I[:, n+1] += (factor * A).dot(I[:, n])

    # Compute error w.r.t to data
    error = 1/2 * ((I - I_hat)**2).sum() + tau/2 * (w**2).sum()
    return error

In [11]:
delta_x = 0.1
delta_t = delta_x**2 / 2
max_x = 20
max_t = 15
x = np.arange(0, max_x + delta_x, delta_x)
t = np.arange(0, max_t + delta_t, delta_t)
size_x = x.size

# Diffusion matrix
A = np.eye(size_x, size_x, 0) * -2 + np.eye(size_x, size_x, -1) + np.eye(size_x, size_x, 1)
# Dyrichlet BC on the left side
A[0][0] = 0
A[0][1] = 0
# Neumann BC on the right side
A[-1][-2] = 2

initial_condition_S = 80
initial_condition_L = 20
initial_condition_I = 0
S = np.zeros((x.size, t.size))
L = np.zeros((x.size, t.size))
I = np.zeros((x.size, t.size))
S[:, 0] = initial_condition_S
L[0, 0] = initial_condition_L
I[0, 0] = initial_condition_I

# From notebook 9: Generate training data
with open('data/training_data_I.npy', 'rb') as f:
    I_hat = np.load(f)

w_hat = np.array([0.39, 0.125, 0.3, 0.01, 0.9, 0.2, 0.5])
w_k0 = np.array([0.5] * w_hat.size)
tau = 0.1

res = minimize(objective, x0=w_k0, args=(tau, S, L, I, I_hat, A, t, delta_x, delta_t))
res.x

C:\Users\Lisandro\AppData\Local\Temp\ipykernel_15192\1765682936.py:10: RuntimeWarning: overflow encountered in multiply
  S[:, n+1] = S[:, n] + delta_t * (b * N - d * S[:, n] - Y * S[:, n] * N - B * S[:, n] * I[:, n])
C:\Users\Lisandro\AppData\Local\Temp\ipykernel_15192\1765682936.py:10: RuntimeWarning: invalid value encountered in subtract
  S[:, n+1] = S[:, n] + delta_t * (b * N - d * S[:, n] - Y * S[:, n] * N - B * S[:, n] * I[:, n])
C:\Users\Lisandro\AppData\Local\Temp\ipykernel_15192\1765682936.py:11: RuntimeWarning: overflow encountered in multiply
  L[:, n+1] = L[:, n] + delta_t * (B * S[:, n] * I[:, n] - d * L[:, n] - Y * L[:, n] * N - e * L[:, n])
C:\Users\Lisandro\AppData\Local\Temp\ipykernel_15192\1765682936.py:12: RuntimeWarning: overflow encountered in multiply
  I[:, n+1] = I[:, n] + delta_t * (e * L[:, n] - Y * I[:, n] * N - a * I[:, n] - d * I[:, n])
C:\Users\Lisandro\AppData\Local\Temp\ipykernel_15192\1765682936.py:19: RuntimeWarning: overflow encountered in square
  e

array([-240.34564311, -385.5727065 ,  206.69216051, -722.78257116,
       -310.10199067,  427.72392513,  134.46587435])

# with solve_ivp

In [3]:
def sli_with_spatial_terms_model(t, y, A, size_x, b, d, B, Y, a, e):
    """
    Parameters
    ----------
    b: Birth rate
    d: Natural death rate
    B: Infection rate
    Y: Parameter to take into account density dependent reduction in population
    a: Death rate due to infection
    e: Rate of individuals that turn infective
    """
    S, L, I = y[:size_x], y[size_x:2*size_x], y[2*size_x:]
    N = S + L + I

    dS_dt = b * N - d * S - Y * S * N - B * S * I
    dL_dt = B * S * I - d * L - Y * L * N - e * L
    dI_dt = e * L - Y * I * N - a * I - d * I
    
    dS_dt += A.dot(S)
    dL_dt += A.dot(L)
    dI_dt += A.dot(I)
    
    return np.concatenate([dS_dt, dL_dt, dI_dt])

In [17]:
def objective(w, tau, S, L, I, I_hat, A, x, t, delta_x, delta_t):
   
    # Organize your data
    b, d, B, Y, a, e, diffusion_rate = w

    # SLI-model step
    A= (diffusion_rate / delta_x**2) * A

    sol = solve_ivp(
        fun=sli_with_spatial_terms_model,
        t_span=[0, t[-1]],
        t_eval=t,
        y0=np.concatenate([S[:,0], L[:,0], I[:,0]]),
        args=(A, x.size, b, d, B, Y, a, e),
        max_step=delta_t
    )
    try:
        S, L, I = sol.y[:x.size,:], sol.y[x.size:2*x.size,:], sol.y[2*x.size:,:]
        # Compute error w.r.t to data
        error = 1/2 * ((I - I_hat)**2).sum() + tau/2 * (w**2).sum()
    except:
        print(t.shape)
        print(S.shape)
        print(L.shape)
        print(I.shape)
        print(I_hat.shape)
        print(w.shape)
        raise Exception
    return error

In [18]:
delta_x = 0.1
delta_t = delta_x**2 / 2
max_x = 20
max_t = 15
x = np.arange(0, max_x + delta_x, delta_x)
t = np.arange(0, max_t + delta_t, delta_t)
size_x = x.size

# Diffusion matrix
A = np.eye(size_x, size_x, 0) * -2 + np.eye(size_x, size_x, -1) + np.eye(size_x, size_x, 1)
# Dyrichlet BC on the left side
A[0][0] = 0
A[0][1] = 0
# Neumann BC on the right side
A[-1][-2] = 2

initial_condition_S = 80
initial_condition_L = 20
initial_condition_I = 0
S = np.zeros((x.size, t.size))
L = np.zeros((x.size, t.size))
I = np.zeros((x.size, t.size))
S[:, 0] = initial_condition_S
L[0, 0] = initial_condition_L
I[0, 0] = initial_condition_I

# From notebook 9: Generate training data
with open('data/training_data_I.npy', 'rb') as f:
    I_hat = np.load(f)

w_hat = np.array([0.39, 0.125, 0.3, 0.01, 0.9, 0.2, 0.5])
w_k0 = np.array([0.5] * w_hat.size)
tau = 0.1

res = minimize(objective, x0=w_k0, args=(tau, S, L, I, I_hat, A, x, t, delta_x, delta_t))
res.x

(3001,)
(201, 10)
(201, 10)
(201, 10)
(201, 3001)
(7,)


Exception: 